### Installing and importing libraries

In [1]:
!pip install hazm
!pip install pandas
!pip install numpy
!pip install nltk
!pip install tqdm
# !pip install transformers
!pip install fasttext

In [2]:
import pandas as pd
import numpy as np
from hazm import *
import tqdm
import json
import codecs
from nltk import FreqDist
import pickle
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# from transformers import BigBirdModel, AutoTokenizer
import fasttext

### Import data

In [67]:
news_dict = {}
with open('../data/news.json', "r", encoding="utf-8") as text_file:
        news_dict = json.loads(text_file.read())

df = pd.DataFrame.from_dict(news_dict, orient='index')
df

,title,subject,content
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی:اصلاحیه قانون به ...
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین،فدا حسین مالکی در مورد ...
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...
...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...


### Pre processing

In [68]:
normalizer = Normalizer()

normalized_news_dict = {}
for idx,value in tqdm.tqdm(news_dict.items()):
    normalized_news_dict[idx] = {"title" : value['title'] , "content":normalizer.normalize(value['content'])}
  
news_df = pd.DataFrame.from_dict(normalized_news_dict, orient='index')

100%|███████████████████████████████████| 14876/14876 [00:12<00:00, 1209.85it/s]


In [69]:
new_stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('../custom_requirments/new_stopwords.txt','r','utf-8').readlines()]
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('../custom_requirments/stopwords.txt','r','utf-8').readlines()]

In [70]:
tokenized_words = [word_tokenize(_) for _ in news_df.content]

In [71]:
removed_tokenized_words = []
for per_content_words in tokenized_words:
  words = []
  for word in per_content_words:
    if word not in stopwords + new_stopwords:
      words.append(word)
  removed_tokenized_words.append(words)

news_df['word_tokenize'] = removed_tokenized_words
news_df

,title,content,word_tokenize
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"[گزارش, همشهری, آنلاین, روزنامه, فرهیختگان, شم..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,"[همشهری‌آنلاین-, امیرمحمدحسینی, اصلاحیه, قانون..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,"[گزارش, همشهری‌آنلاین, فدا, حسین, مالکی, قطعنا..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"[ЩҮЩ…ШҙЩҮШұЫҢ, ШўЩҶЩ„Ш§ЫҢЩҶ, Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"[گزارش, همشهری‌آنلاین, آیت‌الله, سید, ابراهیم,..."
...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[گزارش, همشهری, آنلاین, مسابقات, مقدماتی, جام,..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"[گزارش, همشهری, آنلاین, صعود, پرتغال, لهستان, ..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[گزارش, همشهری, آنلاین, مسابقات, مقدماتی, جام,..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"[گزارش, همشهری, آنلاین, تیم, ملی, مصر, مصاف, س..."


In [72]:
def create_dataframe_least_freq(news_dict):
    news_tokens = {}
    for idx, value in tqdm.tqdm(news_dict.items()):
        news_tokens[idx] = FreqDist(value).most_common()[-10:]
    df = pd.DataFrame.from_dict(news_tokens, orient='index')
    return df.transpose()

def create_dataframe_most_freq(news_dict):
    news_tokens = {}
    for idx, value in tqdm.tqdm(news_dict.items()):
        news_tokens[idx] = FreqDist(value).most_common(10)
    df = pd.DataFrame.from_dict(news_tokens, orient='index')
    return df.transpose()

In [73]:
tokenized_news = {}
for idx,news in news_df.iterrows():
  tokenized_news[idx] = news.word_tokenize

least_freq_df = create_dataframe_least_freq(tokenized_news)
most_freq_df = create_dataframe_most_freq(tokenized_news)

100%|███████████████████████████████████| 14876/14876 [00:01<00:00, 8026.52it/s]


In [74]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt == 1:
        return stemmer.stem(tok)
    if opt == 2:
        return lemmatizer.lemmatize(tok)

In [75]:
opt = 2
lemmatize_stemming_words = []
for per_content_words in news_df.word_tokenize:
  words = []
  for word in per_content_words:
    words.append(get_lemma_set(word,opt))
  lemmatize_stemming_words.append(words)
news_df['word_lemmatize'] = lemmatize_stemming_words

In [76]:
words = []
persian_numbers = ['۱','۲','۳','۴','۵','۶','۷','۸','۹']
for row_words in news_df['word_lemmatize']:
    final_words = ','.join(row_words)
    final_words = final_words.replace('-','')
    for persian_num in persian_numbers:
        if persian_num in list(final_words):
            final_words.replace(persian_num, '')     
    words.append(final_words)
news_df['clean_keyword'] = words
news_df = news_df.drop(['word_tokenize','word_lemmatize'],axis=1)
news_df

,title,content,clean_keyword
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"گزارش,همشهری,آنلاین,روزنامه,فرهیختگان,شماره,ام..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,"همشهری‌آنلاین,امیرمحمدحسینی,اصلاحیه,قانون,کارگ..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,"گزارش,همشهری‌آنلاین,فدا,حسین,مالکی,قطعنامه,ضد,..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"ЩҮЩ…ШҙЩҮШұЫҢ,ШўЩҶЩ„Ш§ЫҢЩҶ,Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,ШӯШ..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"گزارش,همشهری‌آنلاین,آیت‌الله,سید,ابراهیم,رییس,..."
...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"گزارش,همشهری,آنلاین,صعود,پرتغال,لهستان,۵,تیم,آ..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"گزارش,همشهری,آنلاین,تیم,ملی,مصر,مصاف,سنگال,تسا..."


In [77]:
clean_keywords_non_least_frequent = []
for idx,per_final_news in news_df.iterrows():
  for jdx, per_freq in least_freq_df.iterrows():
    try:
      word = least_freq_df.iloc[jdx,int(idx)]
      removed = per_final_news.clean_keyword
      if word is not None:
        word = word[0]
        removed = removed.replace(word,'')
    except Exception as e:
      continue
  clean_keywords_non_least_frequent.append(removed)

clean_keywords_non_most_frequent = []
for idx,per_final_news in news_df.iterrows():
  for jdx, per_freq in most_freq_df.iterrows():
    try:
      word = most_freq_df.iloc[jdx,int(idx)]
      removed = per_final_news.clean_keyword
      if word is not None:
        word = word[0]
        removed = removed.replace(word,'')
    except Exception as e:
      continue
  clean_keywords_non_most_frequent.append(removed)

news_df['clean_keywords_non_least_freq'] = clean_keywords_non_least_frequent
news_df['clean_keywords_non_most_freq'] = clean_keywords_non_least_frequent
news_df

,title,content,clean_keyword,clean_keywords_non_least_freq,clean_keywords_non_most_freq
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"گزارش,همشهری,آنلاین,روزنامه,فرهیختگان,شماره,ام...","گزارش,همشهری,آنلاین,روزنامه,فرهیختگان,شماره,ام...","گزارش,همشهری,آنلاین,روزنامه,فرهیختگان,شماره,ام..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,"همشهری‌آنلاین,امیرمحمدحسینی,اصلاحیه,قانون,کارگ...","همشهری‌آنلاین,امیرمحمدحسینی,اصلاحیه,قانون,کارگ...","همشهری‌آنلاین,امیرمحمدحسینی,اصلاحیه,قانون,کارگ..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,"گزارش,همشهری‌آنلاین,فدا,حسین,مالکی,قطعنامه,ضد,...","گزارش,همشهری‌آنلاین,فدا,حسین,مالکی,قطعنامه,ضد,...","گزارش,همشهری‌آنلاین,فدا,حسین,مالکی,قطعنامه,ضد,..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"ЩҮЩ…ШҙЩҮШұЫҢ,ШўЩҶЩ„Ш§ЫҢЩҶ,Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,ШӯШ...","ЩҮЩ…ШҙЩҮШұЫҢ,ШўЩҶЩ„Ш§ЫҢЩҶ,Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,ШӯШ...","ЩҮЩ…ШҙЩҮШұЫҢ,ШўЩҶЩ„Ш§ЫҢЩҶ,Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,ШӯШ..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"گزارش,همشهری‌آنلاین,آیت‌الله,سید,ابراهیم,رییس,...","گزارش,همشهری‌آنلاین,آیت‌الله,سید,ابراهیم,رییس,...","گزارش,همشهری‌آنلاین,آیت‌الله,سید,ابراهیم,رییس,..."
...,...,...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲...","گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲...","گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"گزارش,همشهری,آنلاین,صعود,پرتغال,لهستان,۵,تیم,آ...","گزارش,همشهری,آنلاین,صعود,پرتغال,لهستان,۵,تیم,آ...","گزارش,همشهری,آنلاین,صعود,پرتغال,لهستان,۵,تیم,آ..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲...","گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲...","گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"گزارش,همشهری,آنلاین,تیم,ملی,مصر,مصاف,سنگال,تسا...","گزارش,همشهری,آنلاین,تیم,ملی,مصر,مصاف,سنگال,تسا...","گزارش,همشهری,آنلاین,تیم,ملی,مصر,مصاف,سنگال,تسا..."


### Inputs and constanst

In [23]:
DOC_RELATED_NUM = 10
query = 'واکسن کرونا' 

### Boolean retrieval

In [24]:
query_words = query.split(" ")
boolean_retrival = [[0 for _ in range(len(query_words))] for __ in range(len(news_df))]
for i in range(len(news_df)):
  keywords = news_df.iloc[i].clean_keyword
  for j,q_word in enumerate(query_words):
      if q_word in keywords.split(","):
          boolean_retrival[i][j] = 1
boolean_df = pd.DataFrame(boolean_retrival, columns = query_words) 
boolean_df

,واکسن,کرونا
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
14871,0,0
14872,0,0
14873,0,0
14874,0,0


In [ ]:
converted_df = boolean_df.all(axis='columns')
converted_df = converted_df.to_frame('res')
boolean_related_docs_index = converted_df.index[converted_df['res'] == True].tolist()
boolean_related_titles = []
for idx in boolean_related_docs_index:
  boolean_related_titles.append(news_df.iloc[idx,0])

In [ ]:
for idx,title in enumerate(boolean_related_titles[:DOC_RELATED_NUM]):
  print(f"{idx+1}   {title}")

1   شرط لازم برای زائران اربعین حسینی(ع) | تداوم لغو روادید زمینی عراق تا اربعین
2   تهمت به روحانیت را تحمل کردیم اما توهین به مقدسات را تحمل نمی‌کنیم
3   تاکید رئیس‌جمهور بر لزوم تشکیل کمیته اجتماعی ستاد ملی مقابله با کرونا
4   سلاح هسته‌ای در دکترین ایران مطلقا جایی ندارد | ظرفیت‌های متنوع مغفول مانده میان ایران و عمان احیا شود
5   ۸۰۰ هزار درخواست مردمی در سفرهای استانی | مشکلات کشور با نشستن در تهران حل نمی‌شود
6   رئیسی: یارانه ۴۰۰ و ۳۰۰ هزار تومانی ماهیانه به هر فرد می‌دهیم | سرنوشت قیمت بنزین؛ نان و دارو گران نمی‌شوند
7   رهبر معظم انقلاب ۱۴ خرداد در حرم مطهر امام راحل سخنرانی می‌کنند
8   تصویر کارت جعلی یارانه نان | واکنش وزارت کشور
9   برخی سیاه نمایی می‌کنند تا در آینده پست بگیرند | بی‌حجابی‌ها چهره شهر شیراز را زشت کرده | سوال اقتصادی امام جمعه موقت شیراز از دولت
10   خبرگزاری روسی: ایران می‌تواند در تامین دارو به روسیه تحت تحریم کمک کند


### TF-IDF retrieval

In [ ]:
vocabulary = set()
for doc in news_df.clean_keyword:
    vocabulary.update(doc.split(','))
vocabulary = list(vocabulary)

In [ ]:
tfidf = TfidfVectorizer(vocabulary=vocabulary, use_idf = True, dtype=np.float32)
tfidf_tran=tfidf.fit_transform([' '.join(doc) for doc in removed_tokenized_words])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


In [ ]:
with open('/content/models/tfidf.pk', 'wb') as f:
    pickle.dump(tfidf, f)

sparse.save_npz("/content/models/tfidf_tran.npz", tfidf_tran)

In [ ]:
with open('/content/models/tfidf.pk', 'rb') as f:
    tfidf_loaded = pickle.load(f)

tfidf_tran_loaded = sparse.load_npz("/content/models/tfidf_tran.npz")

In [ ]:
query_vec = tfidf_loaded.transform([query])
tfidf_results = cosine_similarity(tfidf_tran_loaded,query_vec).reshape((-1,))

In [ ]:

for idx,title in enumerate(tfidf_results.argsort()[-10:][::-1]):
    print(f"{idx+1}   {news_df.iloc[title,0]}")

1   ذخیره ۵۰ میلیون دوز واکسن کرونا در کشور | نیازی به ساخت واکسن‌های داخلیِ بیشتر نیست
2   دپوی ۵۰ میلیون دوز واکسن کرونا در انبارها | ادامه تولید واکسن‌های ایرانی یک تصمیم سیاسی است
3   تصمیم نامشخص وزارت بهداشت درباره جزئیات تزریق دوز چهارم واکسن
4   ممکن است شمار بستری‌ها در موج کرونای اومیکرون دو برابر شود
5   باورهای نادرست در مورد واکسن کرونا| نگاه «همه یا هیچ» نادرست است
6   ۲ شرط برای ادامه کنترل کرونا در ایران | آیا احتمال نیاز به تمدید سالانه واکسن هست؟
7   توصیه وزیر بهداشت درباره تزریق دوز چهارم واکسن کرونا
8   دوز سوم واکسن کرونا، همه چیز درباره واکسن اسپایکوژن
9   چرا تمایل تهرانی‌ها به واکسن زدن در اوج کرونا کم شد؟
10   اشباع بازار جهانی واکسن کرونا| بزرگترین شرکت واکسن‌سازی جهان تولید واکسن آسترازنکا را متوقف می‌کند


### Transformers retrieval

In [ ]:
from transformers import BigBirdModel, AutoTokenizer

MODEL_NAME = "SajjadAyoubi/distil-bigbird-fa-zwnj"
model = BigBirdModel.from_pretrained(MODEL_NAME, block_size=32)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj were not used when initializing BigBirdModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdModel were not initialized from the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj and are newly initialized: ['bert.pooler.

In [ ]:
model.save_pretrained('/content/models/transformer_model.model')
tokenizer.save_pretrained('/content/models/transformer_tokenizer')

('/content/models/transformer_tokenizer/tokenizer_config.json',
 '/content/models/transformer_tokenizer/special_tokens_map.json',
 '/content/models/transformer_tokenizer/vocab.txt',
 '/content/models/transformer_tokenizer/added_tokens.json',
 '/content/models/transformer_tokenizer/tokenizer.json')

In [ ]:
model_loaded = BigBirdModel.from_pretrained('/content/models/transformer_model.model')
tokenizer_loaded = AutoTokenizer.from_pretrained('/content/models/transformer_tokenizer')

In [ ]:
docs_embedding_tran = {}
for idx, doc in enumerate(news_df.clean_keyword):
  text = ' '.join(doc.split(','))
  tokens = tokenizer_loaded(text, return_tensors='pt')
  try:
    output = model_loaded(**tokens)
    print(f"done{idx}")
  except Exception as e:
    text = ' '.join(doc.split(',')[:1000])
    tokens = tokenizer_loaded(text, return_tensors='pt')
    output = model_loaded(**tokens)
  docs_embedding_tran[idx] = output[0][0].detach().numpy()

Attention type 'block_sparse' is not possible if sequence_length: 170 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 352 with config.block_size = 32, config.num_random_blocks = 3. Changing attention type to 'original_full'...
Token indices sequence length is longer than the specified maximum sequence length for this model (619 > 512). Running this sequence through the model will result in indexing errors


done0
done1
done2
done3
done4
done5
done6
done7


KeyboardInterrupt: ignored

In [ ]:
docs_embedding_tran_avg = {}
for k,v in docs_embedding_tran.items():
  sum = np.zeros(768)
  for idx,word_embedding in enumerate(v):
    sum = np.sum([sum, docs_embedding_tran[k][idx]], axis=0)
  docs_embedding_tran_avg[k] = sum /len(docs_embedding_tran[k])

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open('/content/models/transformer_vectors_emb.json', 'w', encoding="utf-8") as f:
    json.dump(docs_embedding_tran_avg, f, cls=NumpyEncoder)

NameError: ignored

In [ ]:
with open('/content/models/transformer_vectors_emb.json', 'r', encoding="utf-8") as f:
  loaded_docs_embs = json.loads(f.read())

In [ ]:
qtokens_tran = tokenizer(query, return_tensors='pt')
qoutput_tran = model(**qtokens_tran)
query_embedding_tran = qoutput_tran[0][0].detach().numpy()
qsum_tran = np.zeros(768)
for qidx,qword in enumerate(query_embedding_tran):
  qsum_tran = np.sum([qsum_tran, np.array(query_embedding_tran[qidx])], axis=0)
q_avg_tran = qsum_tran/len(query_embedding_tran)

Attention type 'block_sparse' is not possible if sequence_length: 4 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 352 with config.block_size = 32, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [ ]:
docs_cosine_similarity_tran = {}
for idx,doc_embedding in docs_embedding_tran_avg.items():
  docs_cosine_similarity_tran[idx] = np.dot(doc_embedding, q_avg_tran) / (np.linalg.norm(doc_embedding)*np.linalg.norm(q_avg_tran))

docs_cosine_similarity_tran = reversed(sorted(docs_cosine_similarity_tran.items(), key=lambda x:x[1]))
sort_docs_cosine_similarity_tran = dict(docs_cosine_similarity_tran)

In [ ]:
for idx, doc_id in enumerate((list(sort_docs_cosine_similarity_tran.items()))[:10]):
  print(f"{idx+1}   {news_df.iloc[doc_id[0]].title}")

1   عکس | روز هشتم آواربرداری در متروپل آبادان
2   آزمایش‌های اولیه واکسن کرونا به صورت قرص خوراکی موفق بوده‌اند
3   واکسن‌های چینی اختصاصی کرونای اُمیکرون آماده کارآزمایی بالینی شدند
4   ذخیره ۵۰ میلیون دوز واکسن کرونا در کشور | نیازی به ساخت واکسن‌های داخلیِ بیشتر نیست
5   ۲ شرط برای ادامه کنترل کرونا در ایران | آیا احتمال نیاز به تمدید سالانه واکسن هست؟
6   عکس | دومین محموله استارلینک به اوکراین رسید | ابتکار جدید در راه‌اندازی
7   بیت‌کوین آینده‌ای ندارد | عقیده نگران‌کننده مدیرعامل یک صرافی دیجیتال
8   تزریق دوز چهارم واکسن کرونا را به این دلایل جدی بگیرید | کدام افراد باید دوز چهارم را بزنند؟
9   چه مقدار ارز دیجیتال راهی اوکراین شده است؟ | نقش رمزارزها در بحران
10   آبله میمونی این عضو را درگیر کند کشنده می‌شود | نیاز به تزریق همگانی واکسن آبله وجود دارد؟


### Fasttext retrieval

In [78]:
tokens_list = news_df.clean_keyword.tolist()
all_news_content = ''
for doc in tokens_list:
    all_news_content += ' '.join(doc.split(",")) + '\n'

In [79]:
with open('../models/fasttext_data.txt', 'w') as text_file:
    text_file.write(all_news_content)

In [80]:
fasttext_model = fasttext.train_unsupervised('../models/fasttext_data.txt', model='skipgram', minCount=4)

Read 3M words
Number of words:  29156
Number of labels: 0
Progress: 100.0% words/sec/thread:   65165 lr:  0.000000 avg.loss:  1.870916 ETA:   0h 0m 0s


In [81]:
fasttext_model.save_model('../models/fasttext.bin')

In [82]:
fasttext_model_loaded = fasttext.load_model('../models/fasttext.bin')

In [19]:
docs_embedding_avg = {}
for idx,doc in enumerate(news_df.clean_keyword):
  sum = np.zeros(100)
  for word in doc.split(','):
    sum = np.sum([sum,fasttext_model_loaded[word]],axis=0)
  docs_embedding_avg[idx] = sum / len(doc.split(','))

In [20]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
        
with open('../models/fasttext_vectors_emb.json', 'w', encoding="utf-8") as f:
    json.dump(docs_embedding_avg, f, cls=NumpyEncoder)

In [21]:
with open('../models/fasttext_vectors_emb.json', 'r', encoding="utf-8") as f:
  loaded_docs_embs_ft = json.loads(f.read())

In [25]:
query_sum = np.zeros(100)
for qword in query_words:
  query_sum = np.sum([query_sum, fasttext_model_loaded[qword]], axis = 0)
query_embedding_avg = query_sum / len(query_words)

In [26]:
docs_cosine_similarity = {}
for idx,doc_embedding in loaded_docs_embs_ft.items():
  docs_cosine_similarity[idx] = np.dot(np.array(doc_embedding), query_embedding_avg) / (np.linalg.norm(np.array(doc_embedding))*np.linalg.norm(query_embedding_avg))

docs_cosine_similarity = reversed(sorted(docs_cosine_similarity.items(), key=lambda x:x[1]))
sort_docs_cosine_similarity = dict(docs_cosine_similarity)

/var/folders/jl/n26nkxfn5fz5lng2zqgww5pr0000gn/T/ipykernel_2582/2360406187.py:3: RuntimeWarning: invalid value encountered in double_scalars
  docs_cosine_similarity[idx] = np.dot(np.array(doc_embedding), query_embedding_avg) / (np.linalg.norm(np.array(doc_embedding))*np.linalg.norm(query_embedding_avg))


In [29]:
sort_docs_cosine_similarity

{'11281': 0.8629755442114146,
 '10795': 0.8513306674244171,
 '10666': 0.838334753292783,
 '5717': 0.8276912960937622,
 '10445': 0.8273654015554679,
 '10802': 0.8249382767568576,
 '5333': 0.8234244312833804,
 '5173': 0.820268342392817,
 '11135': 0.8191886007400498,
 '5470': 0.8172111430436891,
 '10448': 0.8168634957705245,
 '11270': 0.8145941734549614,
 '5263': 0.8118362510834235,
 '4744': 0.8100409966735544,
 '11315': 0.8098877605045254,
 '11206': 0.8098407473046048,
 '4938': 0.8088497536274393,
 '5346': 0.8054249273226429,
 '5104': 0.8049559525185322,
 '10748': 0.8045579288363687,
 '5539': 0.8028620641353318,
 '10918': 0.8025257515751865,
 '10781': 0.8011646966224784,
 '8436': 0.7989541560110143,
 '11218': 0.7967545689834151,
 '10552': 0.7966953283860373,
 '10518': 0.7963499733227924,
 '11265': 0.7955731400698569,
 '10863': 0.7953709414157771,
 '5117': 0.7945429838868183,
 '11029': 0.7933607364440863,
 '10715': 0.7932346795467053,
 '11232': 0.7926216717419317,
 '4948': 0.7916140560357

In [ ]:
for idx, doc_id in enumerate((list(sort_docs_cosine_similarity.items()))[:10]):
  print(f"{idx+1}   {news_df.iloc[int(doc_id[0])].title}")

1   فایزر و بیونتک آزمایش بالینی واکسن مخصوص اُمیکرون را شروع می‌کنند
2   واکسن‌های چینی اختصاصی کرونای اُمیکرون آماده کارآزمایی بالینی شدند
3   دوز چهارم واکسن کرونا بیشتر از دوز سوم ایمنی را افزایش می‌دهد | آیا تزریق دوز چهارم لازم است؟
4   دوز چهارم واکسن کرونا از پیامدهای شدید اُمیکرون جلوگیری می‌کند
5   واکسن جدید شرکت مدرنا بر کرونای اُمیکرون موثر است| راه برای واکسیناسیون سالانه کرونا گشوده می‌شود
6   شرکت مدرنا آزمایش واکسن کرونا اختصاصی اُمیکرون را شروع می‌کند
7   تصمیم نامشخص وزارت بهداشت درباره جزئیات تزریق دوز چهارم واکسن
8   توصیه وزیر بهداشت درباره تزریق دوز چهارم واکسن کرونا
9   اعلام شرایط تزریق دز چهارم واکسن کرونا | چه کسانی باید دز چهارم را تزریق کنند؟
10   آماری فاجعه بار از تعداد ایرانی‌هایی که هنوز دوز اول واکسن کرونا را نزده‌اند!
